# Module 2 Group Assignment
## Voter Fraud

In [1]:
import pandas as pd
import requests

In [2]:
r = requests.get('https://publicinterestlegal.org/county-list', headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'})

### Looking at the data

The file for registration rates contains 3 variables:

**Data Dictionary**
* COUNTY(string): The name of the county
* STATE(string): The name of the state
* REGISTRATION RATE(float): The amount percentage the population that are registered to vote.

In [3]:
regVoters = pd.read_html(r.text, header=0)[0]

In [4]:
regVoters.head()

,COUNTY,STATE,REGISTRATION RATE
0,Lowndes County,Alabama,127%
1,Perry County,Alabama,113%
2,Greene County,Alabama,112%
3,Macon County,Alabama,112%
4,Wilcox County,Alabama,110%


In [5]:
regVoters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   COUNTY             141 non-null    object
 1   STATE              141 non-null    object
 2   REGISTRATION RATE  141 non-null    object
dtypes: object(3)
memory usage: 3.4+ KB


REGISTRATION RATE needs to be converted to a float, since it's labeled as an object.

In [6]:
#convert REGISTRATION RATE to float values
regVoters['REGISTRATION RATE'] = regVoters['REGISTRATION RATE'].str.replace('%', '').astype('float')

Renamed the "%" columns to Obama% and Romney% to represent the number of voters that voter for each participant.


In [7]:
elecdf = pd.read_excel('http://image.guardian.co.uk/sys-files/Guardian/documents/2012/11/14/US_elect_county.xls')
elecdf.rename(columns={'%':'Obama%', '%.1':'Romney%'}, inplace=True)

WARNING *** file size (3825153) not 512 + multiple of sector size (512)


State Postal was changed to State to mirror the STATE column from the voter registration dataframe. To accomplish this, a dictionary of the postal state abbrevations and the state name key value pairs were created.

In [8]:
# Make the state column the same format for both dataframes
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

# replace the state postal name with the full state names
elecdf['State Postal'].replace(states, inplace=True)

In [9]:
elecdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4074 entries, 0 to 4073
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   State Postal  4074 non-null   object 
 1   County Name   4074 non-null   object 
 2   FIPS          4074 non-null   int64  
 3   Obama vote    4074 non-null   int64  
 4   Obama%        4072 non-null   float64
 5   Romney vote   4074 non-null   int64  
 6   Romney%       4072 non-null   float64
dtypes: float64(2), int64(3), object(2)
memory usage: 222.9+ KB


**Data Dictionary**

* State (string): State Name
* County Name(string): Name of county
* FIPS(int): FIPS code for the state
* Obama vote(int): Number of votes for Obama
* Obama%(float): Percentage of voters that voted for Obama
* Romney vote: Number of votes for Romneu
* Romney%: The percentage of voters that voted for Romney

### Calculations
* Calcualate the number of votes that could be fraudulent

$votes * (registration rate-100) = possible fraudulent votes$


### Merging dataframes
The columns used to merge the two datasets are the state and county columns since they are both present in the data. To do this we'll need to remove "County" from the regVoters dataframe to match with the elecdf dataframe.

In [10]:
regVoters['COUNTY'] = regVoters['COUNTY'].str.replace(' County', '')

In [11]:
fraudCounties = pd.merge(regVoters, elecdf, left_on=['STATE', 'COUNTY'], right_on=['State Postal', 'County Name'])

In [12]:
fraudCounties.head()

,COUNTY,STATE,REGISTRATION RATE,State Postal,County Name,FIPS,Obama vote,Obama%,Romney vote,Romney%
0,Lowndes,Alabama,127.0,Alabama,Lowndes,1085,5747,76.4,1754,23.3
1,Perry,Alabama,113.0,Alabama,Perry,1105,4536,74.8,1504,24.8
2,Greene,Alabama,112.0,Alabama,Greene,1063,4514,84.7,799,15.0
3,Macon,Alabama,112.0,Alabama,Macon,1087,9045,87.1,1322,12.7
4,Wilcox,Alabama,110.0,Alabama,Wilcox,1131,4867,74.3,1676,25.6


### Caluclating the amount of possible fraudulent votes

In [13]:
fraudCounties['Total Votes'] = fraudCounties['Obama vote'] + fraudCounties['Romney vote']
fraudCounties['Fraudulent Rate'] = fraudCounties['REGISTRATION RATE'] - 100
fraudCounties['Fraudlent Votes'] = fraudCounties['Total Votes'] * fraudCounties['Fraudulent Rate']

### New Questions To Ask
1. Who are the states with higher fraudulent registrations voting for?
2. Which candidates are being voted for with fraudulent votes?